Heat equation in one spatial dimension:
$$\frac{\partial T}{\partial t} = \alpha \frac{\partial ^2 T}{\partial x^2}$$

Where $\alpha$ is the thermal diffusivity and $T$ is the temperature.

Implicit works differently than what we used.  It will use more data from the "future" in the update including several values of $T$ at $t^{n+1}$.  It will make the scheme more difficult to apply.

## Implicit Schemes

Combine Euler time step with an evaluation of the spatial derivative on the updated solution at $t^{n+1}$.  Discretized gives:

$$\frac{T_i ^{n+1} - T_i ^n}{\Delta t} = \alpha \frac{T_{i+1} ^{n+1} - 2T_i ^{n+1} + T_{i-1} ^{n+1}}{\Delta x^2}$$

From the previous time step we only know $T_i ^n$ but what about $T_i ^{n+1}$, $T_{i-1} ^{n+1}$ and $T_{i+1} ^{n+1}$

Put what we know on the RHS and what we don't know on the LHS, which gives:

$$-T_{i-1} ^{n+1} + \left( 2 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_i ^{n+1} - T_{i+1} ^{n+1} = T_i ^n \frac{\Delta x^2}{\alpha \Delta t}$$

What does it look like with $i = 1$?

$$-T_0 ^{n+1} + \left( 2 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_1 ^{n+1} - T_2 ^{n+1} = T_1 ^n \frac{\Delta x^2}{\alpha \Delta t}$$

and $i = 2$?

$$-T_1 ^{n+1} + \left( 2 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_2 ^{n+1} - T_3 ^{n+1} = T_2 ^n \frac{\Delta x^2}{\alpha \Delta t}$$

What about $i = 3$?

$$-T_2 ^{n+1} + \left( 2 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_3 ^{n+1} - T_4 ^{n+1} = T_3 ^n \frac{\Delta x^2}{\alpha \Delta t}$$

Common element for these 3 at least $T_2 ^{n+1}$

### Boundary Conditions??

The value of $T_0 ^{n+1}$ is known at every time-step from the BC.  All unknown terms on the LHS and known on the RHS for $i = 1$:

$$-T_2 ^{n+1} + \left( 2 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_1 ^{n+1} = T_1 ^n \frac{\Delta x^2}{\alpha \Delta t} + T_0 ^{n+1}$$

For $i = N - 2$:

$$-T_{N-3} ^{n+1} + \left( 2 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_{N-2} ^{n+1} - T_{N-1} ^{n+1} = T_{N-2} ^n \frac{\Delta x^2}{\alpha \Delta t}$$

Discretized Neumann BC on the RHS of the rod is:

$$\frac{T_{N-1} ^n - T_{N-2} ^n}{\Delta x} = q$$

We can easily write that at time step $n + 1$(the boundary conditions apply at every time-step):

$$\frac{T_{N-1} ^{n+1} - T_{N-2} ^{n+1}}{\Delta x} = q$$

Inserting the Neumann BC in the equation for $i = N - 2$ gives:

$$-T_{N-3} ^{n+1} + \left( 1 + \frac{\Delta x^2}{\alpha \Delta t} \right) T_{N-2} ^{n+1} = T_{N-2} ^n \frac{\Delta x^2}{\alpha \Delta t} + \Delta xq$$

Now the linear system of eqs in matrix form are:

$$[A][x] = [b] + [b]_{b.c.}$$

Where the matrix of coefficients $[A]$ is a sparse matrix.